In [5]:
import json

# Load the JSON data
file_path = 'DataEngineeringQ2.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract patient details
patient_details = [entry['patientDetails'] for entry in data]

# Define the columns to check
columns = ['firstName', 'lastName', 'birthDate']

# Initialize missing counts
missing_counts = {col: 0 for col in columns}

# Count missing values
for patient in patient_details:
    for col in columns:
        if not patient.get(col):  # Checking for None or empty string
            missing_counts[col] += 1

# Calculate percentages
total_patients = len(patient_details)
missing_percentages = {col: (count / total_patients) * 100 for col, count in missing_counts.items()}

# Round off the percentages to 2 decimal places
missing_percentages_rounded = {col: round(perc, 2) for col, perc in missing_percentages.items()}

missing_percentages_rounded


{'firstName': 0.0, 'lastName': 70.97, 'birthDate': 32.26}

In [6]:
from collections import Counter
from datetime import datetime

# 2. Percentage of female gender after imputation of gender column basis mode
# Extract gender values
genders = [patient.get('gender') for patient in patient_details if patient.get('gender')]

# Impute missing gender values with mode
mode_gender = Counter(genders).most_common(1)[0][0]
for patient in patient_details:
    if not patient.get('gender'):
        patient['gender'] = mode_gender

# Calculate percentage of female gender
female_count = sum(1 for patient in patient_details if patient.get('gender') == 'F')
female_percentage = (female_count / total_patients) * 100
female_percentage_rounded = round(female_percentage, 2)

# 6. Count of Adult age group
# Add ageGroup column
def calculate_age(birth_date):
    if birth_date:
        birth_date = datetime.strptime(birth_date, "%Y-%m-%dT%H:%M:%S.%fZ")
        return (datetime.now() - birth_date).days // 365
    return None

for patient in patient_details:
    age = calculate_age(patient.get('birthDate'))
    if age is not None:
        if age <= 12:
            patient['ageGroup'] = 'Child'
        elif 13 <= age <= 19:
            patient['ageGroup'] = 'Teen'
        elif 20 <= age <= 59:
            patient['ageGroup'] = 'Adult'
        else:
            patient['ageGroup'] = 'Senior'
    else:
        patient['ageGroup'] = None

adult_count = sum(1 for patient in patient_details if patient.get('ageGroup') == 'Adult')

# 7. Average number of medicines prescribed
medicines_counts = [len(appointment['consultationData']['medicines']) for appointment in data]
average_medicines = sum(medicines_counts) / len(medicines_counts)
average_medicines_rounded = round(average_medicines, 2)

# 8. 3rd most frequently prescribed medicineName
medicine_names = [medicine['medicineName'] for appointment in data for medicine in appointment['consultationData']['medicines']]
medicine_name_counts = Counter(medicine_names)
third_most_common_medicine = medicine_name_counts.most_common(3)[-1][0]

# 9. Percentage distribution of total active and inactive medicines
total_medicines = len(medicine_names)
active_medicines = sum(1 for appointment in data for medicine in appointment['consultationData']['medicines'] if medicine['isActive'])
inactive_medicines = total_medicines - active_medicines
active_percentage = (active_medicines / total_medicines) * 100
inactive_percentage = (inactive_medicines / total_medicines) * 100
active_inactive_percentages_rounded = round(active_percentage, 2), round(inactive_percentage, 2)

# Output all results
missing_percentages_rounded, female_percentage_rounded, adult_count, average_medicines_rounded, third_most_common_medicine, active_inactive_percentages_rounded


({'firstName': 0.0, 'lastName': 70.97, 'birthDate': 32.26},
 32.26,
 21,
 2.13,
 'C',
 (69.7, 30.3))

In [7]:
def is_valid_indian_phone_number(phone_number):
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]

    if phone_number.isdigit() and 6000000000 <= int(phone_number) <= 9999999999:
        return True
    return False

# Add isValidMobile column
for patient in patient_details:
    phone_number = patient.get('phoneNumber')
    patient['isValidMobile'] = is_valid_indian_phone_number(phone_number) if phone_number else False

# Count the number of valid phone numbers
valid_phone_count = sum(1 for patient in patient_details if patient['isValidMobile'])

print(f'Number of valid phone numbers: {valid_phone_count}')


Number of valid phone numbers: 0


In [12]:
import numpy as np
from datetime import datetime

def calculate_age(birth_date):
    if birth_date:
        birth_date = datetime.strptime(birth_date, "%Y-%m-%dT%H:%M:%S.%fZ")
        return (datetime.now() - birth_date).days // 365
    return None

# Extract ages and number of medicines prescribed
ages = []
num_medicines = []

for patient in patient_details:
    age = calculate_age(patient.get('birthDate'))
    if age is not None:
        ages.append(age)
        patient_id = patient.get('patientId')  # Use the correct key here
        num_medicines.append(len(next(appointment['consultationData']['medicines'] for appointment in data if appointment['patientDetails']['patientId'] == patient_id)))

# Calculate Pearson correlation
correlation = np.corrcoef(ages, num_medicines)[0, 1]
correlation_rounded = round(correlation, 2)

print(f'Pearson correlation between number of prescribed medicines and patient\'s age: {correlation_rounded}')


KeyError: 'patientId'